# Tiginagh Section (Modelling)

## Data processing and Augmentation

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models


In [2]:
# Function to load images with quality checks
def load_images(image_folder):
    images = []
    labels = []
    for label in os.listdir(image_folder):
        class_folder = os.path.join(image_folder, label)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            try:
                with Image.open(img_path) as img:
                    # Convert to grayscale and resize
                    img = img.convert('L').resize((28, 28))
                    img_array = np.array(img, dtype=np.float32) / 255.0
                    # Quality control: Check for low contrast images
                    if img_array.std() > 0.05: 
                        images.append(img_array.reshape(28, 28, 1))
                        labels.append(label)
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                continue
    return np.array(images), labels


tifinagh_path = "C:/Users/bouad/OneDrive/Bureau/Amazigh NLP/MNIST-BERBER/Tifinagh-version/imgT"
tifinagh_images, tifinagh_labels = load_images(tifinagh_path)
encoder = LabelEncoder()
tifinagh_labels_encoded = encoder.fit_transform(tifinagh_labels)
num_classes = len(encoder.classes_)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tifinagh_images, tifinagh_labels_encoded, test_size=0.2, random_state=42)

# Data Augmentation
data_augmentation = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

# Training the model with augmented data
train_generator = data_augmentation.flow(X_train, to_categorical(y_train), batch_size=32)


## Capsul Network Model

In [8]:
def squash(x, axis=-1):
    """
    Squashing function to ensure that vectors have magnitudes between 0 and 1.
    """
    s_squared_norm = K.sum(K.square(x), axis=axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * x

def build_capsule_network(input_shape, num_classes):
    input_layer = layers.Input(shape=input_shape)

    # Convolutional layer to extract features
    conv_layer = layers.Conv2D(64, (9, 9), strides=(1, 1), padding='valid', activation='relu')(input_layer)

    primary_caps = layers.Conv2D(32, (9, 9), strides=(2, 2), padding='valid', activation='relu')(conv_layer)
    primary_caps_shape = K.int_shape(primary_caps)
    num_capsules = primary_caps_shape[1] * primary_caps_shape[2] * 32 // 8 

    # Reshaping
    primary_caps = layers.Reshape((num_capsules, 8))(primary_caps)
    primary_caps = layers.Lambda(lambda x: squash(x))(primary_caps)

    digit_caps = layers.Dense(num_classes * 16, activation='relu')(primary_caps)  
    digit_caps = layers.Reshape((-1, num_classes, 16))(digit_caps)
    output_caps = layers.Lambda(lambda x: squash(x))(digit_caps)

    # Calculating the length of output capsules to match the true labels shape
    y_pred = layers.Lambda(lambda z: K.sqrt(K.sum(K.square(z), axis=-1)))(output_caps)  

    # Reducing the dimensionality 
    y_pred = layers.Lambda(lambda x: K.max(x, axis=1))(y_pred)

    # Add Decoder network for reconstruction
    decoder_input = layers.Input(shape=(num_classes, 16))
    flat_caps = layers.Flatten()(decoder_input)

    num_elements = int(np.prod(input_shape))  # Convert to integer

    decoder = models.Sequential([
        layers.Input(shape=(num_classes * 16,)),  
        layers.Dense(512, activation='relu'),
        layers.Dense(1024, activation='relu'),
        layers.Dense(num_elements, activation='sigmoid'),
        layers.Reshape(input_shape)
    ])

    decoder_output = decoder(flat_caps)

    # Adding Full model with two inputs and two outputs
    capsnet = models.Model(inputs=[input_layer, decoder_input], outputs=[y_pred, decoder_output])
    capsnet.compile(optimizer=optimizers.Adam(learning_rate=1e-3), 
                    loss=['categorical_crossentropy', 'mse'], 
                    metrics=[['accuracy'], ['mse']])  

    return capsnet

input_shape = (28, 28, 1)
num_classes = 33  
capsule_model = build_capsule_network(input_shape, num_classes)

# Preparing the training data for the capsule network
X_train_reshaped = X_train.reshape(X_train.shape[0], 28, 28, 1)
y_train_categorical = to_categorical(y_train, num_classes)

decoder_input_data = np.zeros((X_train.shape[0], num_classes, 16))

# Fitting the model using both inputs
capsule_model.fit([X_train_reshaped, decoder_input_data], [y_train_categorical, X_train_reshaped], epochs=10, batch_size=16,
                  validation_data=([X_test.reshape(X_test.shape[0], 28, 28, 1), np.zeros((X_test.shape[0], num_classes, 16))], 
                                   [to_categorical(y_test, num_classes), X_test.reshape(X_test.shape[0], 28, 28, 1)]))



Epoch 1/10
1287/1287 ━━━━━━━━━━━━━━━━━━━━ 110s 69ms/step - lambda_11_accuracy: 0.0330 - loss: nan - sequential_2_mse: 0.1692 - val_lambda_11_accuracy: 0.0293 - val_loss: nan - val_sequential_2_mse: 0.0906
Epoch 2/10
1287/1287 ━━━━━━━━━━━━━━━━━━━━ 83s 64ms/step - lambda_11_accuracy: 0.0303 - loss: nan - sequential_2_mse: 0.0827 - val_lambda_11_accuracy: 0.0293 - val_loss: nan - val_sequential_2_mse: 0.0682
Epoch 3/10
1287/1287 ━━━━━━━━━━━━━━━━━━━━ 83s 64ms/step - lambda_11_accuracy: 0.0288 - loss: nan - sequential_2_mse: 0.0671 - val_lambda_11_accuracy: 0.0293 - val_loss: nan - val_sequential_2_mse: 0.0631
Epoch 4/10
1287/1287 ━━━━━━━━━━━━━━━━━━━━ 84s 65ms/step - lambda_11_accuracy: 0.0300 - loss: nan - sequential_2_mse: 0.0630 - val_lambda_11_accuracy: 0.0293 - val_loss: nan - val_sequential_2_mse: 0.0617
Epoch 5/10
1287/1287 ━━━━━━━━━━━━━━━━━━━━ 81s 63ms/step - lambda_11_accuracy: 0.0301 - loss: nan - sequential_2_mse: 0.0623 - val_lambda_11_accuracy: 0.0293 - val_loss: nan - val_sequ

## Vision Transformer (VIT) Model Using EfficientNetB0

In [25]:
# Resizing images to 32x32
X_train_resized = tf.image.resize(X_train, [32, 32])
X_test_resized = tf.image.resize(X_test, [32, 32])

def build_vit_model(input_shape, num_classes):
    # Using a pre-trained EfficientNetB0 model
    base_model = EfficientNetB0(weights=None, include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output_layer = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

vit_model = build_vit_model((32, 32, 3), num_classes)
vit_model.fit(X_train_resized, to_categorical(y_train), epochs=10, validation_data=(X_test_resized, to_categorical(y_test)))


Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 315s 263ms/step - accuracy: 0.1887 - loss: 2.9656 - val_accuracy: 0.1591 - val_loss: 3.8611
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 226s 300ms/step - accuracy: 0.7482 - loss: 0.8506 - val_accuracy: 0.7900 - val_loss: 0.7541
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 195s 302ms/step - accuracy: 0.8482 - loss: 0.5106 - val_accuracy: 0.8438 - val_loss: 0.5620
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 194s 302ms/step - accuracy: 0.8767 - loss: 0.4390 - val_accuracy: 0.9147 - val_loss: 0.2980
Epoch 5/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 209s 323ms/step - accuracy: 0.8768 - loss: 0.4459 - val_accuracy: 0.1626 - val_loss: 5.7273
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 281s 352ms/step - accuracy: 0.8459 - loss: 0.5357 - val_accuracy: 0.9196 - val_loss: 0.2485
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 226s 350ms/step - accuracy: 0.8751 - loss: 0.4113 - val_accuracy: 0.9567 - val_loss: 0.1771
Epoch 8/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 256s 340ms/step - accuracy: 0.9157 -

## LeNet-5 Model ( CNN ArchitectureType)

In [26]:
def build_lenet5(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(6, (5, 5), activation='tanh', input_shape=input_shape, padding='same'),
        layers.AveragePooling2D(pool_size=(2, 2)),  
        layers.Conv2D(16, (5, 5), activation='tanh'),
        layers.AveragePooling2D(pool_size=(2, 2)),  
        layers.Flatten(),
        layers.Dense(120, activation='tanh'),
        layers.Dense(84, activation='tanh'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

lenet5_model = build_lenet5((28, 28, 1), num_classes)
lenet5_model.fit(X_train_reshaped, y_train_categorical, epochs=10, 
                 validation_data=(X_test.reshape(X_test.shape[0], 28, 28, 1), to_categorical(y_test, num_classes)))


Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.7122 - loss: 1.2352 - val_accuracy: 0.9446 - val_loss: 0.2247
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.9474 - loss: 0.1945 - val_accuracy: 0.9586 - val_loss: 0.1545
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9703 - loss: 0.1153 - val_accuracy: 0.9646 - val_loss: 0.1223
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9852 - loss: 0.0676 - val_accuracy: 0.9662 - val_loss: 0.1115
Epoch 5/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9878 - loss: 0.0521 - val_accuracy: 0.9718 - val_loss: 0.0990
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.9916 - loss: 0.0361 - val_accuracy: 0.9724 - val_loss: 0.0961
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9941 - loss: 0.0268 - val_accuracy: 0.9713 - val_loss: 0.0939
Epoch 8/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.9961 - loss: 0.0184 - val_

## Multi-Layer Perceptron (MLP) Model

In [21]:
def build_mlp(input_shape, num_classes):
    model = models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

mlp_model = build_mlp((28, 28, 1), num_classes)
mlp_model.fit(X_train_reshaped, y_train_categorical, epochs=10, 
              validation_data=(X_test.reshape(X_test.shape[0], 28, 28, 1), to_categorical(y_test, num_classes)))


C:\Users\bouad\anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6475 - loss: 1.4157 - val_accuracy: 0.9031 - val_loss: 0.3249
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9297 - loss: 0.2532 - val_accuracy: 0.9293 - val_loss: 0.2369
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9570 - loss: 0.1592 - val_accuracy: 0.9328 - val_loss: 0.2202
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9695 - loss: 0.1133 - val_accuracy: 0.9431 - val_loss: 0.1942
Epoch 5/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9795 - loss: 0.0775 - val_accuracy: 0.9398 - val_loss: 0.2050
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9846 - loss: 0.0594 - val_accuracy: 0.9443 - val_loss: 0.1919
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9905 - loss: 0.0388 - val_accuracy: 0.9386 - val_loss: 0.2237
Epoch 8/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9918 - loss: 0.0335 - val_accuracy: 0.

## VGG-Like Small Network (CNN Architecture Type)

In [22]:
def build_vgg_like(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

vgg_like_model = build_vgg_like((28, 28, 1), num_classes)
vgg_like_model.fit(X_train_reshaped, y_train_categorical, epochs=10, 
                   validation_data=(X_test.reshape(X_test.shape[0], 28, 28, 1), to_categorical(y_test, num_classes)))


Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 39s 53ms/step - accuracy: 0.7963 - loss: 0.7330 - val_accuracy: 0.9736 - val_loss: 0.0907
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 34s 52ms/step - accuracy: 0.9833 - loss: 0.0563 - val_accuracy: 0.9792 - val_loss: 0.0660
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 33s 51ms/step - accuracy: 0.9894 - loss: 0.0325 - val_accuracy: 0.9823 - val_loss: 0.0610
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 33s 52ms/step - accuracy: 0.9917 - loss: 0.0273 - val_accuracy: 0.9858 - val_loss: 0.0498
Epoch 5/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 36s 55ms/step - accuracy: 0.9977 - loss: 0.0091 - val_accuracy: 0.9800 - val_loss: 0.0776
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 39s 52ms/step - accuracy: 0.9943 - loss: 0.0168 - val_accuracy: 0.9858 - val_loss: 0.0424
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 34s 53ms/step - accuracy: 0.9965 - loss: 0.0110 - val_accuracy: 0.9823 - val_loss: 0.0603
Epoch 8/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 39s 50ms/step - accuracy: 0.9965 - loss: 0.0111 - 

## ResNet (Residual Network) Model

In [27]:
def residual_block(x, filters):
    """
    A residual block that adds a shortcut connection to the output of two convolutional layers.
    If the number of filters changes, a 1x1 convolution is applied to the shortcut to match the shape.
    """
    shortcut = x

    # Main path
    x = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)

    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), padding='same')(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def build_resnet_small(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    
    # Adding residual blocks
    x = residual_block(x, 32)
    x = layers.MaxPooling2D((2, 2))(x)
    x = residual_block(x, 64)  
    x = layers.MaxPooling2D((2, 2))(x)
    
    # Flattening and adding dense layers
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Builduing the model
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

resnet_model = build_resnet_small((28, 28, 1), num_classes)
resnet_model.fit(X_train_reshaped, y_train_categorical, epochs=10, 
                 validation_data=(X_test.reshape(X_test.shape[0], 28, 28, 1), to_categorical(y_test, num_classes)))


Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 51s 68ms/step - accuracy: 0.7896 - loss: 0.7609 - val_accuracy: 0.9738 - val_loss: 0.0957
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 43s 66ms/step - accuracy: 0.9812 - loss: 0.0700 - val_accuracy: 0.9792 - val_loss: 0.0682
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 86s 72ms/step - accuracy: 0.9881 - loss: 0.0359 - val_accuracy: 0.9780 - val_loss: 0.0792
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 46s 72ms/step - accuracy: 0.9926 - loss: 0.0295 - val_accuracy: 0.9883 - val_loss: 0.0466
Epoch 5/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.9930 - loss: 0.0234 - val_accuracy: 0.9782 - val_loss: 0.0801
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 42s 65ms/step - accuracy: 0.9962 - loss: 0.0149 - val_accuracy: 0.9864 - val_loss: 0.0580
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 42s 65ms/step - accuracy: 0.9970 - loss: 0.0121 - val_accuracy: 0.9862 - val_loss: 0.0585
Epoch 8/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 83s 67ms/step - accuracy: 0.9964 - loss: 0.0107 - 